In [134]:
import datetime
import json
import pandas as pd
import requests
import sqlalchemy
import pandas.io.sql as sqlio 
import psycopg2 as ps 

f = open("credentials.json")
credentials = json.load(f)

my_path = list(credentials.values())[0]
weather_key = list(credentials.values())[1]
aws_access_key = list(credentials.values())[2]
aws_secret_key = list(credentials.values())[3]
rds_host = list(credentials.values())[4]
rds_user = list(credentials.values())[5]
rds_password = list(credentials.values())[6]
rds_database = list(credentials.values())[7]
rds_charset = list(credentials.values())[8]
rds_port = list(credentials.values())[9]

In [ ]:
# Connecting to AWS PostgreSQL DB
try:
    conn2 = ps.connect(dbname = rds_database, 
                       user = rds_user, 
                       password = rds_password, 
                       host = rds_host, 
                       port = rds_port)
    conn2.autocommit = True
    cur = conn2.cursor()
    print('Connection established.')
except:
    print("Error while connecting to PostgreSQL.")

In [ ]:
# Request current weather data
location = pd.DataFrame({'city_name': ['Berlin', 'Tulum', 'Ko Tao', 'Parque Nacional Corcovado', 'San Diego'], 'latitude': [52.520007, 20.211419, 10.095610, 8.540835, 32.715738 ] , 'longitude': [13.404954, -87.465350, 99.840396, -83.570964, -117.161084]})

openweathermap_api_key = weather_key

def current_weather(row):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={row.latitude}&lon={row.longitude}&appid={openweathermap_api_key}"
    response = requests.get(url)
    response_json = response.json()
    date = datetime.datetime.fromtimestamp(response_json["dt"])
    today = date.today()
    today_year = today.year
    return {
        "date_column": date,
        "main.temp": response_json["main"]["temp"] - 273.15,
        "rain.1h": response_json.get("rain", {}).get("1h", 0.0),
        "Y": today_year
    }

df_weather = location.apply(lambda x: current_weather(x), axis=1)
current_weather = pd.concat([location, pd.json_normalize(df_weather)], axis=1)
current_weather = current_weather.drop(['latitude', 'longitude'], axis=1)
print(current_weather)

In [ ]:
# Create RDS table for current weather
try:
    cur.execute("create table if not exists weather_current (city_name varchar(255) primary key, date_column timestamp, \"main.temp\" float8, \"rain.1h\" float8, Y int, CONSTRAINT weather_current_unique_constraint UNIQUE (city_name, date_column))")
    print("Table created successfully")
except:
    print("Error while creating table")

# Insert current weather data into table
try:
    for i, row in current_weather.iterrows():
        cur.execute("insert into weather_current (city_name, date_column, \"main.temp\", \"rain.1h\", Y) values (%s, %s, %s, %s, %s)", (
            row['city_name'], row['date_column'], row['main.temp'], row['rain.1h'], row['Y']))
        print("Data inserted successfully")
except:
    print("Error while inserting data into table")

conn2.commit()    
cur.close()
conn2.close()